# IMPORT STATEMENTS

In [1]:
!TMPDIR=/home/mgill/ pip install --cache-dir=/home/mgill/ --build /home/mgill/ pandas numpy sklearn xgboost matplotlib tensorflow==2.1.0 keras==2.2.4 fastai python-dateutil scikit-optimize scikit-learn==0.21 graphviz

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
import tensorflow as tf; print(tf.__version__)
import keras; print(keras.__version__)
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.regularizers import *
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
#from fastai.tabular.all import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from random import randint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Activation
from math import sqrt
from statistics import mean
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt 
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve, auc
import skopt
from skopt.searchcv import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import GridSearchCV
import pickle
from collections import OrderedDict
from sklearn.model_selection import StratifiedKFold
from scipy import interp
from sklearn.impute import SimpleImputer

2.1.0


Using TensorFlow backend.


2.2.4


# Load in Data, Prepare OHE, Prepare Main Data

In [3]:
#delete the 2 phenos in the files
#only 3 samples, won't create an accurate model, so its better to treat it as a binary classification
def del_2(vcf, pheno):
    i = 0
    rm_idx = []
    print(pheno.shape)
    print(vcf.shape)
    while (i < pheno.shape[0]):
        if (pheno[i] == 2.0):
            print("found 2")
            rm_idx.append(i)
        i = i + 1
    pheno = np.delete(pheno, rm_idx, 0)
    vcf = vcf.drop((vcf.index[rm_idx]))
    print(pheno.shape)
    print(vcf.shape)
    return vcf, pheno

In [4]:
def new_prep_data(tt_file, ho_file):
    imp = SimpleImputer(missing_values='./.', strategy='most_frequent')
    my_list = []
    x = 0 
    for chunk in pd.read_csv(tt_file, chunksize=10000, index_col="Unnamed: 0"):
        x=x+10000
        chunk = chunk.T
        if 'Value' in chunk.columns:
            #does the selecting of pheno array for application ML
            chunk["Value"] = pd.to_numeric(chunk["Value"], downcast="float")
            tt_pheno = chunk["Value"].to_numpy()
            #reshapes it so its not a 1D array
            print(tt_pheno.shape)
            tt_pheno = np.reshape(tt_pheno,(len(tt_pheno),1))
            print(tt_pheno.shape)
            chunk = chunk.drop(columns=['Value'])
        headers = chunk.columns
        row_idx = chunk.index
        chunk = imp.fit_transform(chunk) #SHOULD TURN ./. into the most common for each column
        #since imputing makes a numpy array have to turn back into PD for label encoding
        chunk = pd.DataFrame(data = chunk, index = row_idx, columns = headers)
        my_list.append(chunk)
        print(x)
    tt_vcf = pd.concat(my_list, axis = 1)
    my_list = []
    x=0
    for chunk in pd.read_csv(ho_file, chunksize=10000, index_col="Unnamed: 0"):
        x=x+10000
        chunk = chunk.T
        if 'Value' in chunk.columns:
            #does the selecting of pheno array for application ML
            chunk["Value"] = pd.to_numeric(chunk["Value"], downcast="float")
            ho_pheno = chunk["Value"].to_numpy()
            #reshapes it so its not a 1D array
            print(ho_pheno.shape)
            ho_pheno = np.reshape(ho_pheno,(len(ho_pheno),1))
            print(ho_pheno.shape)
            chunk = chunk.drop(columns=['Value'])
        headers = chunk.columns
        row_idx = chunk.index
        chunk = imp.fit_transform(chunk) #SHOULD TURN ./. into the most common for each column
        #since imputing makes a numpy array have to turn back into PD for label encoding
        chunk = pd.DataFrame(data = chunk, index = row_idx, columns = headers)
        my_list.append(chunk)
        print(x)
    ho_vcf = pd.concat(my_list, axis = 1)
    tt_vcf,tt_pheno = del_2(tt_vcf, tt_pheno)
    ho_vcf,ho_pheno = del_2(ho_vcf, ho_pheno)
    print(tt_vcf.shape)
    print(ho_vcf.shape)
    print(tt_pheno.shape)
    print(ho_pheno.shape)
    return tt_vcf, ho_vcf, tt_pheno, ho_pheno

In [ ]:
tt_vcf, ho_vcf, tt_pheno, ho_pheno = new_prep_data("PuD_Merged_filtered.csv_train_test.csv_5pcnt.csv", "PuD_Merged_filtered.csv_holdout.csv_5pcnt.csv")

In [ ]:
#only if it hasn't been done yet otherweise skip to load
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
ohe = ohe.fit(tt_vcf)

In [ ]:
pickle.dump(ohe, open("PuD_ohe.dat", "wb"))

In [ ]:
#if need or have new holdout data etc.
ohe = pickle.load(open("PuD_ohe.dat", "rb"))

In [ ]:
print(tt_vcf.shape)
tt_vcf = ohe.transform(tt_vcf)
print(tt_vcf.shape)
print(ho_vcf.shape)
ho_vcf = ohe.transform(ho_vcf)
print(ho_vcf.shape)

In [ ]:
def find_snp_from_header(ohe,snp_num):
    count = 0
    snp = "Not found"
    found = False
    i = 0
    while i < len(ohe.categories_) and (found == False):
        j = 0
        while j < len(ohe.categories_[i]):
            if(count == snp_num):
                snp = ohe.categories_[i][j]
                found = True
                break
            count = count + 1
            j = j + 1
        i = i + 1
    return snp

In [ ]:
## TESTING IF IT WORKS
my_snp = find_snp_from_header(ohe, 10805)
print(my_snp)

# XGBoost ML Model

### Importing the model building data, one hot encoding and creating a label set

In [ ]:
#for optimising the parameters, dont need to run if going straight to k-fold eval
print(tt_vcf.shape)
print(tt_pheno.shape)
seed = randint(0,5000)
X_train, X_test, y_train, y_test = train_test_split(tt_vcf, tt_pheno, test_size=0.2, random_state=seed)
print(X_test.shape)
print(X_train.shape)
print("seed is " + str(seed))

# Setting Space to Optimise

In [ ]:

space ={'learning_rate': Real(0.01, 1.0, 'log-uniform'),
        'min_child_weight': Integer(0, 10),
        'max_depth': Integer(0, 50),
        'max_delta_step': Integer(0, 20),
        'subsample': Real(0.01, 1.0, 'uniform'),
        'colsample_bytree': Real(0.01, 1.0, 'uniform'),
        'colsample_bylevel': Real(0.01, 1.0, 'uniform'),
        'reg_lambda': Real(1e-9, 1000, 'log-uniform'),
        'reg_alpha': Real(1e-9, 1.0, 'log-uniform'),
        'gamma': Real(1e-9, 0.5, 'log-uniform'),
        'min_child_weight': Integer(0, 5),
        'n_estimators': Integer(50, 200),
        'scale_pos_weight': Real(1e-6, 500, 'log-uniform')}

# Stops the optimising if above 98% accuracy

In [ ]:
def on_step(optim_result):
    """
    Callback meant to view scores after
    each iteration while performing Bayesian
    Optimization in Skopt"""
    score = xgb_bayes_search.best_score_
    print("best score: %s" % score)
    if score >= 0.98:
        print('Interrupting!')
        return True

# Look for optimium parameters from the defined space and print best params

In [ ]:
xgbcl = xgb.XGBClassifier()
xgb_bayes_search = BayesSearchCV(xgbcl, space, n_iter=32, # specify how many iterations
                                    scoring=None, n_jobs=1, cv=5, verbose=3, random_state=42, n_points=12,
                                 refit=True)
xgb_bayes_search.fit(X_train, y_train.ravel(), callback = on_step)
print(xgb_bayes_search.best_params_)

# Take Best Model from the Optimisation

In [ ]:
#either print out from the optimisation or run the ordered dict.
#print(xgb_bayes_search.best_params_)
best_params = OrderedDict([('colsample_bylevel', 1.0), ('colsample_bytree', 1.0), ('gamma', 0.07059904760306446), ('learning_rate', 0.011073929920126888), ('max_delta_step', 20), ('max_depth', 29), ('min_child_weight', 4), ('n_estimators', 52), ('reg_alpha', 1e-09), ('reg_lambda', 1000.0), ('scale_pos_weight', 0.2860271078350677), ('subsample', 1.0)])

In [ ]:
model = xgb.XGBClassifier(**xgb_bayes_search.best_params_)

In [ ]:
model.fit(X_train, y_train.ravel(), verbose=True)

# Use this model to predict the test set and evaluate

In [ ]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
#sees how accurate the model was when testing the test set
accuracy = accuracy_score(y_test, predictions)
pcent = accuracy * 100.0
print("The accuracy of this model is" + str(pcent))
xgb_predictions = model.predict(X_test)
xgb_probs = model.predict_proba(X_test)[:, 1]

In [ ]:
#Evaluation Function to create ROC Score and AUROC Graph
#from matplotlib.pyplot import figure
#figure(num=None, figsize=(10, 10), dpi=100, facecolor='w', edgecolor='k')
def evaluate_model(predictions, probs, y_pheno):
   #Computes statistics and shows ROC curve.
    
    results = {}
    
    results['recall'] = recall_score(y_pheno, predictions)
    results['precision'] = precision_score(y_pheno, predictions)
    results['roc'] = roc_auc_score(y_pheno, probs)
    my_roccy = ("(ROC SCORE: %.2f)" % (roc_auc_score(y_pheno, probs)))
    
    # Calculate false positive rates and true positive rates
    model_fpr, model_tpr, _ = roc_curve(y_pheno, probs)
    return model_fpr, model_tpr, my_roccy

    #plt.figure(figsize = (8, 6))
   # plt.rcParams['font.size'] = 16
    
    # Plot both curves
   # plt.plot(model_fpr, model_tpr, 'r', label = 'Holdout Data'+my_roccy)
   # plt.legend();
   # plt.xlabel('False Positive Rate'); 
   # plt.ylabel('True Positive Rate'); plt.title('AUROC Curve for Xgboost Classification of Flower Colour');
   # plt.show();

In [ ]:
evaluate_model(xgb_predictions, xgb_probs)

# 10-fold cross validation on training / holdout data, evaluation and AUROC

# Creating Model from best parameters and then evaluating

In [ ]:
def eval_k_fold(m, x, y, k, hx, hy):
    #model: xgboost model, should be with the best params available
    #x: input data (eg. all samples and SNPS)
    #y: labels
    #k: number of folds for cross validation
    cv = StratifiedKFold(n_splits=k,shuffle=False)
    fig1 = plt.figure(figsize=[12,12])

    tprs = []
    aucs = []
    results = []
    mean_fpr = np.linspace(0,1,100)
    high = 0
    best = m
    i = 1
    for train,test in cv.split(x,y):
        prediction = m.fit(x[train],y[train].ravel()).predict_proba(x[test])
        print("variables for auroc curve done. Processing fold accuracy + checking best model")
        y_pred = m.predict(x[test])
        predictions = [round(value) for value in y_pred]
        #sees how accurate the model was when testing the test set
        accuracy = accuracy_score(y[test], predictions)
        pcent = accuracy * 100.0
        print("The accuracy of this model is" + str(pcent))
        if(pcent > high):
            high = pcent
            best = m
        fpr, tpr, t = roc_curve(y[test], prediction[:, 1])
        tprs.append(interp(mean_fpr, fpr, tpr))
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        results.append(pcent)
        plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
        i= i+1

    plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
    mean_tpr = np.mean(tprs, axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)
    plt.plot(mean_fpr, mean_tpr, color='blue',
             label=r'Mean ROC (AUC = %0.2f )' % (mean_auc),lw=2, alpha=1)
    
    holdout_pred = best.predict(hx)
    predictions = [round(value) for value in holdout_pred]
    #sees how accurate the model was when testing the test set
    accuracy = accuracy_score(hy, predictions)
    pcent = accuracy * 100.0
    print(pcent)
    xgb_predictions = best.predict(hx)
    xgb_probs = best.predict_proba(hx)[:, 1]
    model_fpr, model_tpr, my_roccy = evaluate_model(xgb_predictions, xgb_probs, hy)
    plt.plot(model_fpr, model_tpr, 'r', label = 'Holdout Data'+my_roccy, lw=2)
    
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Pubescence Density Training Model & Holdout Data')
    plt.legend(loc="lower right")
    plt.show()

    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    print("Holdout Accuracy: %.2f%%" % (pcent))
    
    return best

In [ ]:
#THESE ARE THE BEST PARAMETERS
best_params = xgb_bayes_search.best_params_
#best_params = OrderedDict([('colsample_bylevel', 1.0), ('colsample_bytree', 1.0), ('gamma', 0.07059904760306446), ('learning_rate', 0.011073929920126888), ('max_delta_step', 20), ('max_depth', 29), ('min_child_weight', 4), ('n_estimators', 52), ('reg_alpha', 1e-09), ('reg_lambda', 1000.0), ('scale_pos_weight', 0.2860271078350677), ('subsample', 1.0)])
#load holdout data
print(tt_vcf.shape)
print(tt_pheno.shape)
print(ho_vcf.shape)
print(ho_pheno.shape)
model_2 = xgb.XGBClassifier(**best_params) #if optimised in same session, other enter manually below
#this function should average out 10 folds and training, with inital params optimised
#average accuracy and std should be calculated along with a nice AUROC graph of train/test models
#best model should be extracted for use on holdout set
best_model = eval_k_fold(model_2, tt_vcf, tt_pheno, 10, ho_vcf, ho_pheno)


# save this model

In [ ]:
import pickle
pickle.dump(best_model, open("PuD_kfold_10_XGB.dat", "wb"))

# load model

In [ ]:
#only load if not generated in same session
best_model = pickle.load(open("PuD_kfold_10_XGB.dat", "rb"))

# SNPS of Importance

In [ ]:
from xgboost import plot_importance
from matplotlib import pyplot
#best_model = pickle.load(open("FC_kfold_10_tt_from_all.pickle.dat", "rb"))
plt.figure(figsize = (20, 20))
plot_importance(best_model, max_num_features=15, importance_type='gain', height=0.3)
pyplot.show()

##WTF???

In [ ]:
#This function essentially returns an array of dataframe headers the length of OHE'd input SNPs for training data
#EG. It will be able to determine that feature 357310 is Gm13_17683957 but not what allele it is
#eg. feature 357309 357310 and 357311 may all be one hot encoded versions of all possible values of Gm13_17683957
#iterating through the saved OHE will by able to determine what specific allele the feature is but cannot determine
#what SNP header it belongs to. Therefore combining these two methods you can determine both allele and SNP
snp = []
imp = SimpleImputer(missing_values='./.', strategy='most_frequent')
fs_ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
x = 0
n_headers = []
le = LabelEncoder()
#while (i < 10):
for chunk in pd.read_csv("PuD_Merged_filtered.csv_train_test.csv_5pcnt.csv", chunksize=10000, index_col="Unnamed: 0"):
    chunk = chunk.T
    if 'Value' in chunk.columns:
        print("dropping value so it doesn't include that in headers")
        chunk = chunk.drop(columns=['Value'])
    headers = chunk.columns
    row_idx = chunk.index
    chunk = imp.fit_transform(chunk) #SHOULD TURN ./. into the most common for each column
    #since imputing makes a numpy array have to turn back into PD for label encoding
    chunk = pd.DataFrame(data = chunk, index = row_idx, columns = headers)
    chunk = chunk.apply(lambda col: le.fit_transform(col))
    c_headers = chunk.columns
    y = 0
    for column in chunk:
        d = (chunk[column].nunique())
        n_headers.extend([c_headers[y] for i in range(d)])
        #print(n_headers)
        #print(l)
        #n_headers.append(c_headers[y] * d)
        #print(n_headers)
        y = y + 1
    #to double check that it would indeed be one hot encoded with this amount of columns
    chunk = fs_ohe.fit_transform(chunk)
    x = x + chunk.shape[1]
    print("my X value is: " + str(x))
    print(chunk.shape)
    print("my header list is: " + str(len(n_headers)))
print(len(n_headers))

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(10, 10), dpi=100, facecolor='w', edgecolor='k')
fs = [343112]
scores = [3.2]
snp_label = []
for jj in fs:
    jj_allele = find_snp_from_header(ohe, jj)
    this_snp = (n_headers[jj] + ' ('+str(jj_allele)+')')
    print(this_snp)
    snp_label.append(this_snp)
snp_label.reverse()
scores.reverse()
print(len(scores))
print(len(snp_label))
plt.barh(snp_label,scores)
plt.title('SNP Importance XGBoost Pubescence Density')
plt.ylabel('SNP Label')
plt.xlabel('Relative F_Score (GAIN)')
plt.show()

In [ ]:
my_dict = best_model.get_booster().get_score(importance_type="gain")

In [ ]:
print(my_dict)

In [ ]:
def rename_f_header(fn,n_headers,ohe):
    fn = fn[1:]
    fn = int(fn)
    allele = find_snp_from_header(ohe, fn)
    this_snp = (n_headers[fn] + ' ('+str(allele)+')')
    return this_snp

In [ ]:
#convert feature to actual SNP name
i = 0
new_dict = {}
for key in my_dict:
    new_key = rename_f_header(key, n_headers, ohe)
    new_dict[new_key] = my_dict[key]
    i = i + 1
    print(str(i))
    if(my_dict):
        continue
    else:
        break

In [ ]:
new_fi = pd.Series(new_dict)
print(new_fi)
df = new_fi.to_frame()
df = df.rename(columns = {0:'F_Score(GAIN)'})
print(df)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(10, 10), dpi=100, facecolor='w', edgecolor='k')
indexes = df.nlargest(20, "F_Score(GAIN)").index
values = df.nlargest(20, "F_Score(GAIN)").values.ravel()
indexes = indexes[::-1]
values = values[::-1]
plt.barh(indexes, values)
plt.title('SNP Importance XGBoost Pubescence Density')
plt.ylabel('SNP Label')
plt.xlabel('Relative F_Score (GAIN)')
plt.show()

In [ ]:
#generate figure object
figure(num=None, figsize=(10, 10), dpi=100, facecolor='w', edgecolor='k')
#load in the 20 lardest values and their SNP label
indexes = df.nlargest(20, "F_Score(GAIN)").index
values = df.nlargest(20, "F_Score(GAIN)").values.ravel()
#reverse to make the largest be at the front
indexes = indexes[::-1]
values = values[::-1]
#for each different chromosome you want to colour add a index(*_i) and value (*_v) array
#black would be colour for singular/notinteresting chromosomes
r_i = []
r_v = []
b_i = []
b_v = []
g_i = []
g_v = []
y_i = []
y_v = []
bl_i = []
bl_v = []
p_i = []
p_v = []
br_i = []
br_v = []
pu_i = []
pu_v = []
#for each value in the top n (default 20) check which chromosome it belongs to and add it to the colour array
i = 0
while i < len(indexes):
    if('Gm08' in indexes[i]):
        r_i.append(indexes[i])
        r_v.append(values[i])
    elif('Gm12' in indexes[i]):
        b_i.append(indexes[i])
        b_v.append(values[i])
    elif('Gm01' in indexes[i]):
        g_i.append(indexes[i])
        g_v.append(values[i])
    #elif('Gm11' in indexes[i]):
    #    y_i.append(indexes[i])
    #    y_v.append(values[i])
    #elif('Gm08' in indexes[i]):
    #    p_i.append(indexes[i])
    #    p_v.append(values[i])
   # elif('Gm04' in indexes[i]):
   #     br_i.append(indexes[i])
   #     br_v.append(values[i])
   # elif('Gm13' in indexes[i]):
   #     pu_i.append(indexes[i])
   #     pu_v.append(values[i])
    else:
        bl_i.append(indexes[i])
        bl_v.append(values[i])
    i = i + 1
#plot each of the arrays with appropriate colour and label graph
plt.barh(bl_i, bl_v, color="black")
plt.barh(br_i, br_v, color="brown")
plt.barh(pu_i, pu_v, color="purple")
plt.barh(y_i, y_v, color="yellow")
plt.barh(p_i, p_v, color="orange")
plt.barh(g_i, g_v, color="green")
plt.barh(r_i, r_v, color="red")
plt.barh(b_i, b_v, color="blue")
plt.title('SNP Importance XGBoost Pubescence Density')
plt.ylabel('SNP Label')
plt.xlabel('Relative F_Score (GAIN)')
plt.show()

# Random Forest ML Model

In [ ]:
tt_vcf, ho_vcf, tt_pheno, ho_pheno = new_prep_data('PuD_Merged_filtered.csv_train_test.csv_5pcnt.csv', 'PuD_Merged_filtered.csv_holdout.csv_5pcnt.csv')

In [ ]:
ohe = pickle.load(open("PuD_ohe.dat", "rb"))
tt_vcf = ohe.transform(tt_vcf)
print(tt_vcf.shape)
print(ho_vcf.shape)
ho_vcf = ohe.transform(ho_vcf)
print(ho_vcf.shape)

In [ ]:
seed = randint(0,5000)
rf_model = RandomForestClassifier(n_estimators=100, random_state=seed, max_features = 'sqrt',n_jobs=-1, verbose = 1)
best_rf_model = eval_k_fold(rf_model, tt_vcf, tt_pheno, 10, ho_vcf, ho_pheno)

In [ ]:
import pickle
pickle.dump(best_rf_model, open("PuD_kfold_10_RF.dat", "wb"))

In [ ]:
feat_importances = pd.Series(best_model.feature_importances_, index=n_headers)
feat_importances.nlargest(20).plot(kind='barh')

# CNN Model (based upon DL Primer)
### one hot encode and train test split like you usually do but follow the primer for the actual model part
### for BC and MC use primer, for regression use Philipps notebooks....

In [ ]:
tt_vcf, ho_vcf, tt_pheno, ho_pheno = new_prep_data('PuD_Merged_filtered.csv_train_test.csv_5pcnt.csv', 'PuD_Merged_filtered.csv_holdout.csv_5pcnt.csv')

In [ ]:
ohe = pickle.load(open("PuD_ohe.dat", "rb"))
tt_vcf = ohe.transform(tt_vcf)
print(tt_vcf.shape)
print(ho_vcf.shape)
ho_vcf = ohe.transform(ho_vcf)
print(ho_vcf.shape)

In [ ]:
def build_CNN_model(x_len):    
    #del model
    model = Sequential()
    model.add(Conv1D(filters=12, kernel_size=14, 
                     input_shape=(x_len, 1)))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Conv1D(filters=10, kernel_size=10, 
                     input_shape=(12, 1)))
    model.add(Activation('linear'))
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=8, kernel_size=8, 
                     input_shape=(10, 1)))
    model.add(Activation('linear'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(48, activation='linear'))
    model.add(Dense(32, activation='linear'))
    model.add(Dense(16, activation='linear'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))
    opt = tf.keras.optimizers.Adamax(learning_rate=0.003)#, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"


    model.compile(loss='binary_crossentropy', optimizer=opt, 
                  metrics=['binary_accuracy'])

    print(model.summary())
    return model

In [ ]:
def eval_cnn(x,y,k):
    cv = StratifiedKFold(n_splits=k,shuffle=False)
    best_model = []
    results = []
    highest = 0
    i = 1
    for train,test in cv.split(x,y):
        x = x.reshape(x.shape[0], x.shape[1], 1)
        model = build_CNN_model(x[train].shape[1])
        bs = ((x[train].shape[0])/40)
        bs = round(bs)
        history = model.fit(x[train], y[train], validation_data=(x[test], y[test]), epochs=100, batch_size=bs)
        _, accuracy = model.evaluate(x[test], y[test], batch_size=bs, verbose=0)
        accuracy = accuracy *100
        print("accuracy for model " + str(i) + " is " + str(accuracy))
        if(accuracy > highest):
            highest = accuracy
            best_model = model
        results.append(accuracy)
        del model
        i = i + 1
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results))) 
    return best_model

In [ ]:
best_CNN = eval_cnn(tt_vcf, tt_pheno, 10)

In [ ]:
bs = ((ho_vcf.shape[0])/40)
bs = round(bs)
ho_vcf = ho_vcf.reshape(ho_vcf.shape[0], ho_vcf.shape[1],1)
_, accuracy = best_CNN.evaluate(ho_vcf, ho_pheno, batch_size=bs, verbose=0)
print("Holdout accuracy is " + str(accuracy*100))

In [ ]:
import pickle
pickle.dump(best_CNN, open("PuD_kfold_10_CNN.dat", "wb"))

# DNN Model

In [5]:
tt_vcf, ho_vcf, tt_pheno, ho_pheno = new_prep_data('PuD_Merged_filtered.csv_train_test.csv_5pcnt.csv', 'PuD_Merged_filtered.csv_holdout.csv_5pcnt.csv')

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
(617,)
(617, 1)
220000
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
(155,)
(155, 1)
220000
(617, 1)
(617, 214900)
found 2
found 2
(615, 1)
(615, 214900)
(155, 1)
(155, 214900)
found 2
(154, 1)
(154, 214900)
(615, 214900)
(154, 214900)
(615, 1)
(154, 1)


In [6]:
ohe = pickle.load(open("PuD_ohe.dat", "rb"))
tt_vcf = ohe.transform(tt_vcf)
print(tt_vcf.shape)
print(ho_vcf.shape)
ho_vcf = ohe.transform(ho_vcf)
print(ho_vcf.shape)

(615, 623421)
(154, 214900)
(154, 623421)


In [7]:
#My own DNN model based upon paper
#del model #incase its stored a previous model
#del history #for redoing shit

#do batch size as 64
#reduce the inputs by half when you read it in
#add XGboost and RF to the one notebook
def build_DNN_model(x_len):
    model = Sequential()

    #add first input layer, with no normalization
    model.add(Dense(256, input_dim = x_len))
    model.add(Activation('relu'))
    model.add(Dropout(0.03))
    
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.02))
    
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.01))
    
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    
    model.add(Dense(16))
    model.add(Activation('relu'))
    #add output layer
    model.add(Dense(1, activation='sigmoid'))
    opt = tf.keras.optimizers.Adamax(learning_rate=0.003)

    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['binary_accuracy'])
    print(model.summary())
    return model



In [8]:
def eval_dnn(x,y,k): 
    cv = StratifiedKFold(n_splits=k,shuffle=False)
    best_model = []
    results = []
    highest = 0
    i = 1
    for train,test in cv.split(x,y):
        model = build_DNN_model(x[train].shape[1])
        bs = ((x[train].shape[0])/40)
        bs = round(bs)
        history = model.fit(x[train], y[train], validation_data=(x[test], y[test]), epochs = 100, batch_size=bs)
        _, accuracy = model.evaluate(x[test], y[test], batch_size=bs, verbose=0)
        accuracy = accuracy * 100
        print("accuracy for model " + str(i) + " is " + str(accuracy))
        if(accuracy > highest):
            highest = accuracy
            best_model = model
        results.append(accuracy)
        i = i + 1
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results))) 
    return best_model

In [9]:
best_DNN = eval_dnn(tt_vcf, tt_pheno, 10)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               159596032 
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8

553/553 [==============================] - 2s 4ms/sample - loss: 0.0315 - binary_accuracy: 0.9892 - val_loss: 0.8086 - val_binary_accuracy: 0.8226
Epoch 38/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0066 - binary_accuracy: 0.9982 - val_loss: 0.5484 - val_binary_accuracy: 0.8548
Epoch 39/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0064 - binary_accuracy: 0.9982 - val_loss: 0.6051 - val_binary_accuracy: 0.8387
Epoch 40/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0050 - binary_accuracy: 1.0000 - val_loss: 0.5553 - val_binary_accuracy: 0.8387
Epoch 41/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0064 - binary_accuracy: 1.0000 - val_loss: 0.7185 - val_binary_accuracy: 0.8387
Epoch 42/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0036 - binary_accuracy: 1.0000 - val_loss: 0.6682 - val_binary_accuracy: 0.8548
Epoch 43/100
553/553 [==============================]

553/553 [==============================] - 2s 4ms/sample - loss: 0.0067 - binary_accuracy: 0.9982 - val_loss: 1.2903 - val_binary_accuracy: 0.8226
Epoch 89/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0058 - binary_accuracy: 0.9982 - val_loss: 1.6968 - val_binary_accuracy: 0.6774
Epoch 90/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0054 - binary_accuracy: 0.9982 - val_loss: 1.2674 - val_binary_accuracy: 0.8065
Epoch 91/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0096 - binary_accuracy: 0.9982 - val_loss: 0.6922 - val_binary_accuracy: 0.8548
Epoch 92/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0033 - binary_accuracy: 1.0000 - val_loss: 1.0443 - val_binary_accuracy: 0.8065
Epoch 93/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0212 - binary_accuracy: 0.9928 - val_loss: 1.2325 - val_binary_accuracy: 0.7581
Epoch 94/100
553/553 [==============================]

553/553 [==============================] - 2s 4ms/sample - loss: 0.0214 - binary_accuracy: 0.9928 - val_loss: 0.4133 - val_binary_accuracy: 0.8065
Epoch 25/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0157 - binary_accuracy: 0.9946 - val_loss: 0.4748 - val_binary_accuracy: 0.8871
Epoch 26/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0181 - binary_accuracy: 0.9964 - val_loss: 0.5626 - val_binary_accuracy: 0.8387
Epoch 27/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0146 - binary_accuracy: 0.9964 - val_loss: 0.4058 - val_binary_accuracy: 0.8871
Epoch 28/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0131 - binary_accuracy: 0.9982 - val_loss: 0.4284 - val_binary_accuracy: 0.8387
Epoch 29/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0172 - binary_accuracy: 0.9946 - val_loss: 0.5374 - val_binary_accuracy: 0.8548
Epoch 30/100
553/553 [==============================]

553/553 [==============================] - 2s 4ms/sample - loss: 0.0150 - binary_accuracy: 0.9928 - val_loss: 0.9472 - val_binary_accuracy: 0.8226
Epoch 76/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0051 - binary_accuracy: 1.0000 - val_loss: 0.6806 - val_binary_accuracy: 0.8387
Epoch 77/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0035 - binary_accuracy: 1.0000 - val_loss: 0.8176 - val_binary_accuracy: 0.8387
Epoch 78/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0035 - binary_accuracy: 0.9982 - val_loss: 0.6880 - val_binary_accuracy: 0.8387
Epoch 79/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0057 - binary_accuracy: 0.9982 - val_loss: 1.4669 - val_binary_accuracy: 0.8387
Epoch 80/100
553/553 [==============================] - 2s 4ms/sample - loss: 0.0047 - binary_accuracy: 0.9982 - val_loss: 0.6815 - val_binary_accuracy: 0.8387
Epoch 81/100
553/553 [==============================]

...

Epoch 52/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.0072 - binary_accuracy: 0.9964 - val_loss: 1.6097 - val_binary_accuracy: 0.6230
Epoch 53/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.0053 - binary_accuracy: 1.0000 - val_loss: 1.2608 - val_binary_accuracy: 0.7705
Epoch 54/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.0011 - binary_accuracy: 1.0000 - val_loss: 1.2933 - val_binary_accuracy: 0.7541
Epoch 55/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.0086 - binary_accuracy: 0.9964 - val_loss: 1.2903 - val_binary_accuracy: 0.8361
Epoch 56/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.0236 - binary_accuracy: 0.9928 - val_loss: 2.1920 - val_binary_accuracy: 0.5246
Epoch 57/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.0046 - binary_accuracy: 0.9982 - val_loss: 1.2308 - val_binary_accuracy: 0.7705
Epoch 58/100
554/554 [==================

Train on 554 samples, validate on 61 samples
Epoch 1/100
554/554 [==============================] - 11s 21ms/sample - loss: 0.5648 - binary_accuracy: 0.7040 - val_loss: 1.0247 - val_binary_accuracy: 0.8197
Epoch 2/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.4250 - binary_accuracy: 0.8177 - val_loss: 0.4104 - val_binary_accuracy: 0.7705
Epoch 3/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.3716 - binary_accuracy: 0.8357 - val_loss: 0.3909 - val_binary_accuracy: 0.8197
Epoch 4/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.3368 - binary_accuracy: 0.8736 - val_loss: 0.5729 - val_binary_accuracy: 0.6885
Epoch 5/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.2743 - binary_accuracy: 0.8917 - val_loss: 0.4768 - val_binary_accuracy: 0.8197
Epoch 6/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.2041 - binary_accuracy: 0.9314 - val_loss: 0.7254 - val_binary_accuracy: 0.8197

Epoch 52/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.0013 - binary_accuracy: 1.0000 - val_loss: 0.9180 - val_binary_accuracy: 0.8525
Epoch 53/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.0021 - binary_accuracy: 1.0000 - val_loss: 0.8293 - val_binary_accuracy: 0.8689
Epoch 54/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.0041 - binary_accuracy: 0.9982 - val_loss: 0.6640 - val_binary_accuracy: 0.8852
Epoch 55/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.0016 - binary_accuracy: 1.0000 - val_loss: 1.1182 - val_binary_accuracy: 0.8361
Epoch 56/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.0131 - binary_accuracy: 0.9964 - val_loss: 0.6961 - val_binary_accuracy: 0.8689
Epoch 57/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.0037 - binary_accuracy: 1.0000 - val_loss: 0.8531 - val_binary_accuracy: 0.8033
Epoch 58/100
554/554 [==================

Train on 554 samples, validate on 61 samples
Epoch 1/100
554/554 [==============================] - 11s 21ms/sample - loss: 0.5001 - binary_accuracy: 0.8032 - val_loss: 2.1035 - val_binary_accuracy: 0.8197
Epoch 2/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.4126 - binary_accuracy: 0.8177 - val_loss: 1.2826 - val_binary_accuracy: 0.8197
Epoch 3/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.3971 - binary_accuracy: 0.8285 - val_loss: 1.4830 - val_binary_accuracy: 0.8197
Epoch 4/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.3228 - binary_accuracy: 0.8502 - val_loss: 0.9062 - val_binary_accuracy: 0.8197
Epoch 5/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.2674 - binary_accuracy: 0.8809 - val_loss: 0.5933 - val_binary_accuracy: 0.8197
Epoch 6/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.2185 - binary_accuracy: 0.9170 - val_loss: 0.9161 - val_binary_accuracy: 0.8197

Epoch 52/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.0048 - binary_accuracy: 1.0000 - val_loss: 1.4687 - val_binary_accuracy: 0.7705
Epoch 53/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.0062 - binary_accuracy: 0.9982 - val_loss: 1.1663 - val_binary_accuracy: 0.7705
Epoch 54/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.0085 - binary_accuracy: 0.9982 - val_loss: 1.1725 - val_binary_accuracy: 0.7541
Epoch 55/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.0131 - binary_accuracy: 0.9964 - val_loss: 0.8449 - val_binary_accuracy: 0.7705
Epoch 56/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.0065 - binary_accuracy: 0.9982 - val_loss: 1.3046 - val_binary_accuracy: 0.7869
Epoch 57/100
554/554 [==============================] - 2s 4ms/sample - loss: 0.0164 - binary_accuracy: 0.9946 - val_loss: 1.1004 - val_binary_accuracy: 0.7377
Epoch 58/100
554/554 [==================

In [10]:
bs = ((ho_vcf.shape[0])/40)
bs = round(bs)
_, accuracy = best_DNN.evaluate(ho_vcf, ho_pheno, batch_size=bs, verbose=0)
print("Holdout accuracy is " + str(accuracy*100))

Holdout accuracy is 80.51947951316833


In [ ]:
best_model = []
results = []
highest = 0
i = 1
while(i<6):
    seed = randint(0,5000)
    #divide up the training and testing data here
    X_train, X_test, y_train, y_test = train_test_split(tt_vcf, tt_pheno, test_size=0.2, random_state=seed)
    #del my_list #to save memory
    print(X_train.shape)
    print(X_test.shape)
    print(y_train.shape)
    print(y_test.shape)


    #might need to reshape it??
    #reshape into (examples, input snps, 1)
    #A is x_train, B is x_test
    # I think this is traditionally because CNNs are image data which use 3 dimensional input data?
    print(X_train.shape)
    print(X_test.shape)
    model = build_DNN_model()
    bs = ((X_train.shape[0])/20)
    bs = round(bs)
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=bs)
    _, accuracy = model.evaluate(X_test, y_test, batch_size=bs, verbose=0)
    print("accuracy for model " + str(i) + " is " + str(accuracy))
    if(accuracy > highest):
        highest = accuracy
        best_model = model
    results.append(accuracy)
    i = i + 1
print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results*100), np.std(results*100))) 

In [ ]:
print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results)*100, np.std(results)*100)) 
bs = ((ho_vcf.shape[0])/20)
bs = round(bs)
_, accuracy = model.evaluate(ho_vcf, ho_pheno, batch_size=bs, verbose=0)
print("Holdout accuracy is " + str(accuracy*100))


In [ ]:
model = my_DNN()
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=64)

In [ ]:
_, accuracy = model.evaluate(X_test, y_test, batch_size=16, verbose=0)

In [ ]:
accuracy

In [ ]:
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
plt.figure()
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
estimator = KerasClassifier(build_fn=my_DNN, epochs=50, batch_size=64, verbose=1)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, vcf, pheno, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))